In [1]:
import sys
sys.path.append('../..')
import numpy as np
# PyTorch
import torch
import torch.nn.functional as F
from torch import cuda
from torch.autograd import Variable
from torchvision.models.mobilenet import mobilenet_v2

# DLBio and own scripts
from DLBio.pytorch_helpers import get_device
import transforms_data as td
from helpers import cuda_conv
import metrics as M
import acquire_ear_dataset as a

from PIL import Image
import glob

In [2]:
class Config():
    DEVICE = get_device()
    DATASET_DIR = '../dataset/'
    AUTH_DATASET_DIR = '../auth_dataset/unknown-auth/'
    MODEL_DIR = './models/model_MN_color.pt'
    RESIZE_SMALL = False
    DATABASE_FOLDER = './embeddings/'

In [3]:
model = torch.load(Config.MODEL_DIR, map_location=torch.device(Config.DEVICE))
preprocess = td.transforms_siamese_verification( td.get_resize(Config.RESIZE_SMALL) )

In [4]:
def image_pipeline(input_, preprocess):
    #input_ = input_.convert("L")
    input_ = preprocess(input_)
    input_ = input_.reshape(-1, td.get_resize(Config.RESIZE_SMALL)[0], td.get_resize(Config.RESIZE_SMALL)[1], 1)
    input_ = input_.permute(3, 0, 1, 2)

    if cuda.is_available():
        return input_.type('torch.cuda.FloatTensor')
    else:
        return input_.type('torch.FloatTensor')

In [5]:
result_value = []
result_label = []

img = Image.open(Config.DATASET_DIR + 'janna_qua/janna_qua003.png')
new_embedding = model(Variable(image_pipeline(img,preprocess))).cpu()

for label in os.listdir(Config.DATABASE_FOLDER):
    loaded_embeddings = np.load(Config.DATABASE_FOLDER+label, allow_pickle=True)
    tmp = []    
    for embedding in loaded_embeddings:
        dis = F.pairwise_distance(embedding,new_embedding)
        tmp.append(dis.item())
    result_value.append((min(tmp)))
    result_label.append(label)


In [7]:
result_value, result_label = zip(*sorted(zip(result_value, result_label)))
result_value = result_value[:10]
result_label = result_label[:10]

for idx, val in enumerate(result_label):
    print(str(idx+1) + ' : ' + ' ' + val + ' : ' + ' ' + str(result_value[idx]))

1 :  janna_qua.npy :  3.577686220523901e-05
2 :  tim_moe.npy :  0.09579121321439743
3 :  clara_pau.npy :  0.09974296391010284
4 :  collin_sch.npy :  0.11074979603290558
5 :  marina_fri.npy :  0.1114228218793869
6 :  clemens_blu.npy :  0.11395391076803207
7 :  beatrix_mah.npy :  0.11544628441333771
8 :  gregor_spi.npy :  0.11710245907306671
9 :  mohammed_muh.npy :  0.11872308701276779
10 :  konrad_von.npy :  0.11907434463500977
